<a href="https://colab.research.google.com/github/IntelligenceX-ai/ix-language-bertsum-extractive-summarization/blob/feature-first_implementation/Model/bertsum_extractive_summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Extractive Summarization with BERT**

Reference: https://skimai.com/tutorial-how-to-fine-tune-bert-for-summarization/